In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load dataset
data = pd.read_csv("Churn_Modelling.csv")

# Separate features and target
X = data.drop(columns=[data.columns[-1]])
y = data[data.columns[-1]]

# Encode categorical columns
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Define ANN model
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()

        # Hyperparameters:
        # 2 hidden layers for sufficient learning capacity
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, len(torch.unique(y_train)))

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.out(x)

# Initialize model, loss, optimizer
model = ANN(X_train.shape[1])

criterion = nn.CrossEntropyLoss()   # taking cross entropy as loss function
optimizer = optim.Adam(model.parameters(), lr=0.001) # taking adam as optimizer

# 10. Training loop
epochs = 100
for epoch in range(epochs):
    model.train()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 11. Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, preds = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test.numpy(), preds.numpy())

print("Test Accuracy:", accuracy)


Epoch [10/100], Loss: 0.5081
Epoch [20/100], Loss: 0.3913
Epoch [30/100], Loss: 0.3523
Epoch [40/100], Loss: 0.3228
Epoch [50/100], Loss: 0.2978
Epoch [60/100], Loss: 0.2751
Epoch [70/100], Loss: 0.2538
Epoch [80/100], Loss: 0.2327
Epoch [90/100], Loss: 0.2116
Epoch [100/100], Loss: 0.1901
Test Accuracy: 0.772
